In [1]:
from bs4 import BeautifulSoup
import re
import sqlite3
import glob

# Парсинг person

In [2]:
database = "../db/data.db"
resumes = glob.glob("../data/resumes/*.html")

def insert_person_data(conn, data):
    sql = ''' INSERT INTO person (age, gender, town, key_profession, specialtys, key_skills, months_of_exp)
              VALUES (?, ?, ?, ?, ?, ?, ?) '''
    cur = conn.cursor()
    cur.execute(sql, data)
    conn.commit()
    return cur.lastrowid

In [18]:
all_resumes = []
for resume in resumes:
    with open(resume, 'r', encoding='utf-8') as file:
        html_content = file.read()
    all_resumes.append(BeautifulSoup(html_content, 'html.parser'))

In [19]:
for i in range(len(all_resumes)):
    soup = all_resumes[i]

    # -------------------------
    # Пол, возраст, город
    gender_element = soup.find('span', {'data-qa': 'resume-personal-gender'})
    gender = gender_element.text if gender_element else None
    
    age_element = soup.find('span', {'data-qa': 'resume-personal-age'})
    age_text = age_element.text if age_element else None
    
    city_element = soup.find('span', {'data-qa': 'resume-personal-address'})
    city = city_element.text if city_element else None
    
    # Используем регулярные выражения для извлечения числа из строки возраста
    if age_text:
        age = int(re.search(r'\d+', age_text).group())
    else:
        age = None
    
    # -------------------------
    # Специализации
    specializations_elements = soup.find_all('li', class_='resume-block__specialization')
    
    specializations = [element.text.strip() for element in specializations_elements]
    
    specializations = ', '.join(specializations)

    # -------------------------
    # Опыт работы
    experience_elements = soup.find_all('span', class_='resume-block__title-text_sub')[0].find_all('span')
    if not experience_elements:
        experience_elements = soup.find_all('span', class_='resume-block__title-text_sub')[1].find_all('span')

    years = 0
    months = 0
    
    for element in experience_elements:
        text = element.text.strip()
        # Проверяем, содержит ли текст информацию о годах
        if 'лет' in text or 'год' in text:
            years = int(re.search(r'\d+', text).group())
    
        # Проверяем, содержит ли текст информацию о месяцах
        elif 'месяц' in text or 'месяцев' in text:
            months = int(re.search(r'\d+', text).group())
    
    total_months = years * 12 + months

    # -------------------------
    # Ключевые навыки
    if "Ключевые навыки" in html_content:
        # Находим блок с ключевыми навыками
        skills_block = soup.find('div', class_='bloko-tag-list')
        
        # Находим все элементы с ключевыми навыками
        skill_elements = skills_block.find_all('div', {'data-qa': 'bloko-tag bloko-tag_inline'})
        
        # Извлекаем текст каждого навыка
        skills = [skill.find('span', {'data-qa': 'bloko-tag__text'}).text.strip() for skill in skill_elements]
        
        skills = ', '.join(skills)
    else:
        skills = None

    # -------------------------
    # Основная профессия
    profession_element = soup.find('span', class_='resume-block__title-text')
    profession = profession_element.text.strip()
    profession

    # -------------------------
    # Запись в БД
    conn = sqlite3.connect(database)
    
    data = (age, gender, city, profession, specializations, skills, total_months)
    insert_person_data(conn, data)

    conn.close()

(27, 'Мужчина', 'Ростов-на-Дону', 'Ветеринарный врач', 'Ветеринарный врач', 'Ветеринария', 64)
(63, 'Мужчина', 'Москва', 'Ведущий инженер-механик', 'Механик, Сервисный инженер, инженер-механик', 'организация ремонта, Организация совещаний, Управление производственным персоналом', 449)
(44, 'Женщина', 'Тамбов', 'Управляющий, администратор', 'Другое, Агроном, Администратор магазина, администратор торгового зала', 'Русский — Родной', 206)
(64, 'Мужчина', 'Ставрополь', 'Главный инженер, инженер по эксплуатации', 'Сервисный инженер, инженер-механик, Инженер по эксплуатации', 'Работоспособность', 568)
(36, 'Мужчина', 'Лиски', 'Зоотехник по кормлению КРС', 'Зоотехник', '1С, Dc, Dp, Triolet, Ecopoint, MS Excel, MS Word', 54)
(43, 'Мужчина', 'Анапа', 'Инженер-механик', 'Сервисный инженер, инженер-механик, Механик', 'Русский — Родной, Немецкий — A1 — Начальный', 217)
(61, 'Мужчина', 'Гигант', 'Инженер-механик', 'Сервисный инженер, инженер-механик, Механик', 'Русский — Родной', 226)
(33, 'Женщина

# Парсинг образования

In [20]:
def process_html(html_content, soup, person_id):
    is_high = 1 if "Высшее образование" in html_content else 0
    education_blocks = soup.find_all('div', {'data-qa': 'resume-block-education-item'})

    if not education_blocks:
        # Обработка случаев, когда блоки образования отсутствуют
        return [(person_id, None, None, is_high)]
    
    education_data = []
    for block in education_blocks:
        university_element = block.find('div', {'data-qa': 'resume-block-education-name'})
        university_name = university_element.text.strip() if university_element else 'Не указано'

        specialization_element = block.find('div', {'data-qa': 'resume-block-education-organization'})
        specialization_text = specialization_element.text.strip() if specialization_element else 'Не указано'

        education_data.append((person_id, university_name, specialization_text, is_high))
    
    return education_data


def main():
    with sqlite3.connect(database) as conn:
        cursor = conn.cursor()
        all_data = []

        for i in range(len(all_resumes)):
            soup = all_resumes[i]
            education_data = process_html(html_content, soup, i + 1)
            all_data.extend(education_data)
        cursor.executemany("INSERT INTO education (person_id, name, specialitys, is_high_edu) VALUES (?, ?, ?, ?)", all_data)
        conn.commit()

if __name__ == "__main__":
    main()

[(1, None, None, 1)]
[(2, 'Московский государственный агроинженерный университет им. В.П. Горячкина, Москва', 'Технический сервис в АПК, Механизация сельского хозяйства', 1), (2, 'курсы', 'ВС, Ремонт и обслуживание аэродромной техники', 1)]
[(3, 'ТОГБПОУ "АГРАРНО-ТЕХНОЛОГИЧЕСКИЙ ТЕХНИКУМ"', 'Агрономия, Агроном', 1), (3, 'Современная гуманитарная академия, Москва', 'Юридический', 1)]
[(4, 'ФГБОУ ВО "Невинномысский государственный технический институт"', 'Профессиональной переподготовки, Управленческий менеджмент', 1), (4, 'Академия, завод "Ростсельмаш"', 'повышение квалификации, Новый модельный ряд сельскохозяйственной техники', 1), (4, 'ФГБОУ ВО "Ставропольский государственный аграрный университет""', 'Профессиональной переподготовки, ПД в учреждениях СПО', 1), (4, 'Ставропольский ордена Трудового Красного Знамени сельскохозяйственный институт', 'Факультет механизации сельского хозяйства, инженер-механик', 1)]
[(5, 'Курская государственная сельскохозяйственная академия им.  И.И. Иванов

# Парсинг jobs

In [5]:
def insert_job_data(conn, data):
    sql = ''' INSERT INTO jobs (job_name, person_id, desc, start, end)
              VALUES (?, ?, ?, ?, ?) '''
    cur = conn.cursor()
    cur.execute(sql, data)
    conn.commit()
    return cur.lastrowid

In [22]:
for i in range(len(all_resumes)):
    soup = all_resumes[i]
    
    # Найти все блоки с опытом работы
    experience_blocks = soup.find_all("div", class_="bloko-column bloko-column_xs-4 bloko-column_s-2 bloko-column_m-2 bloko-column_l-2")
    experience_blocks2 = soup.find_all("div", class_="bloko-column bloko-column_xs-4 bloko-column_s-6 bloko-column_m-7 bloko-column_l-10")
    
    parsed_dates = []
    for block in experience_blocks:
        # Извлекаем текст и проверяем наличие разделителя дат
        text = block.get_text(separator=" ", strip=True)
    
        if " — " in text:
            # Разделяем текст на дату начала и дату окончания
            start_date, end_date = text.split(" — ")
            # Очищаем дату окончания от дополнительной информации
            end_date = " ".join(end_date.split()[:2])
        else:
            # Если разделитель дат отсутствует, устанавливаем дату начала и None для даты окончания
            start_date = text
            end_date = None
        if end_date != None:
            if end_date == 'по настоящее':
                end_date = None
            parsed_dates.append((start_date, end_date))
    
    #print(parsed_dates)
    
    job_descriptions = []
    for block in experience_blocks2:
        # Название профессии
        title_element = block.find("div", {"data-qa": "resume-block-experience-position"})
        job_title = title_element.get_text(strip=True) if title_element else None
    
        # Описание профессии
        description_element = block.find("div", {"data-qa": "resume-block-experience-description"})
        job_description = description_element.get_text(strip=True) if description_element else None
        if job_title != None and job_description != None:
            job_descriptions.append((job_title, job_description))
    
    for dates, jobs in zip(parsed_dates, job_descriptions):
        data = (jobs[0], i + 1, jobs[1], dates[0], dates[1])
        conn = sqlite3.connect(database)
        insert_job_data(conn, data)
        conn.close()